# **NLP4_Run3**

In [1]:
!pip install openai
!pip install python-dotenv
!pip3 install pyomo
!apt install glpk-utils
!pip install glpk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 74.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 5.9 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
Suggested packages:
  libiodbc2-dev
The following NEW packages will be installed:
  glpk-utils libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
0 upgraded, 5 newly installed, 0 to remove and 45 not upgraded.
Need to get 625 kB of archives.
After this operation, 2,158 kB of additional disk space will be used.
Ge

In [2]:

import openai
import os
from IPython.display import Markdown


### **Accessing the GPT4 API**

In [3]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv('api_file.env'))
my_api_key = os.environ['api_key_env']
openai.api_key = my_api_key


### **Generate Mathematical Model**

In [4]:
problem = """We are looking at an alkylation process which will include the following 10 variables: olefin feed (barrels per day), isobutane recycle (barrels per day), acid addition rate (thousands of pounds per day), alkylate yield (barrels per day), isobutane makeup (barrels per day), acid strength (weight per cent), motor octane number, external isobutane-to-olefin ratio, acid dilution factor and F-4 performance number.

We want to maximize the daily profit of this alkylation process.
The profit is defined as the revenue generated from the alkylate yield multiplied with the motor octane number, minus the operational costs, which include olefin feed, isobutane recycle, acid addition rate, and isobutane makeup.

Relationships in terms of other variables for alkylate yield, motor octane number, acid dilution factor, and F-4 performance number can be formulated as regression formulas.
This regression estimate can deviate in both directions from true value of these variables by 2, 1, 5 and 10 percent, respectively.
Alkylate yield is a function of olefin feed and external isobutane-to-olefine yield. Alkalyte yield equals the amount of olefin feed multiplied by the sum of 1.12, 0.13167 times the external isobutane-to-olefin ratio and -0.00667 times the external isobutane-to-olefin ratio squared.
The motor octane number is derived from the external isobutane-to-olefin ratio and the acid strength. The motor octane number is calculated as the sum of 86.35, 1.098 time external isobutane-to-olefin ratio, -0.038 times the external isobutane-to-olefin ratio squared and 0.325 times acid strength reduced by 89.
The acid dilution factor is calculated based on the F-4 performance number. The acid dillution factor is expressed as 35.82 minus 0.222 times F-4 performance number.
Lastly, the F-4 performance number depends on the motor octane number. F-4 performance number is calculated as -133 plus three times the motor octane number.

There are some additional constraints imposed by the nature of the chemical process.
Each variable has a lower and an upper bound.
The external isobutane-to-olefin ratio needs to equal the ratio of isobutane recycle plus isobutane makeup to olefin feed.
The acid strength needs to equal the ratio of 93000 times acid addition rate to acid addition rate multiplied by acid dilution factor in addition to 1000 times acid addition rate.
Lastly, 1.22 alkylate yield needs to be equal to the combined olefin feed and isobutane makeup."""

In [5]:

client = openai.OpenAI(api_key=os.environ['api_key_env'])

response1 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate only the variables for this mathematical optimization problem."},
        {
            "role": "user",
            "content": problem
    }
    ],
    model="gpt-4",
    seed = 3
)


In [6]:
print(response1.choices[0].message.content)

1. Decision variables:
    - \(X_1\): Olefin feed (barrels/day) 
    - \(X_2\): Isobutane recycle (barrels/day)
    - \(X_3\): Acid addition rate (thousands of pounds/day)
    - \(X_4\): Isobutane makeup (barrels/day)
    - \(X_5\): External isobutane-to-olefin ratio
    - \(X_6\): Acid dilution factor
    
2. Other involved variables:
    - \(Y_1\): Alkylate yield (barrels/day)
    - \(Y_2\): Motor octane number
    - \(Y_3\): F-4 performance number
    - \(Y_4\): Acid strength (weight per cent)

3. Constraint variables through regression relationships:
    - \(a_1\): 1.12 + 0.13167 \(X_5\) - 0.00667 \({X_5}^{2}\) to estimate \(Y_1\)
    - \(a_2\): 86.35 + 1.098 \(X_5\) - 0.038 \({X_5}^{2}\) + 0.325 (\(Y_4\) - 89) to estimate \(Y_2\)
    - \(a_3\): 35.82 - 0.222 \(Y_3\) to estimate \(X_6\)
    - \(a_4\): -133 + 3 \(Y_2\) to estimate \(Y_3\)

4. Constraints to ensure the nature of chemical process:
    - \(X_5\) = \(\frac{(X_2 + X_4)}{X_1}\)
    - \(Y_4\) = \(\frac{93000X_3}{(X_3 * X_6

In [7]:

response2 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate only the objective function for this mathematical optimization problem."},
        {
            "role": "user",
            "content": problem + response1.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 3
)

In [8]:
print(response2.choices[0].message.content)

Maximize Z = price*(Y1*Y2) - cost_olefin*(X1) - cost_isobutane_recycle*(X2) - cost_acid*(X3) - cost_isobutane_makeup*(X4)


In [9]:
response3 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate only the constraints for this mathematical optimization problem."},
        {
            "role": "user",
            "content": problem + response1.choices[0].message.content +  response2.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 3
)

In [10]:
print(response3.choices[0].message.content)

1. Constraints from regression relationships:

   Y1 ≤ 1.02*(X1 * (1.12 + 0.13167*X5 - 0.00667*X5^2))
   Y1 ≥ 0.98*(X1 * (1.12 + 0.13167*X5 - 0.00667*X5^2))
   
   Y2 ≤ 1.01*(86.35 + 1.098*X5 - 0.038*X5^2 + 0.325*(Y4 - 89))
   Y2 ≥ 0.99*(86.35 + 1.098*X5 - 0.038*X5^2 + 0.325*(Y4 - 89))
   
   X6 ≤ 1.05*(35.82 - 0.222*Y3)
   X6 ≥ 0.95*(35.82 - 0.222*Y3)
   
   Y3 ≤ 1.1*(-133 + 3*Y2)
   Y3 ≥ 0.9*(-133 + 3*Y2)
   
2. Constraints from chemical process nature:

    X5 = (X2 + X4)/X1
    Y4 = 93000*X3/(X3 * X6 + 1000*X3)
    1.22*Y1 = X1 + X4

3. Lower and upper bounds for each decision variables:

    LB1 ≤ X1 ≤ UB1
    LB2 ≤ X2 ≤ UB2
    LB3 ≤ X3 ≤ UB3
    LB4 ≤ X4 ≤ UB4
    LB5 ≤ X5 ≤ UB5
    LB6 ≤ X6 ≤ UB6

Where, LB is lower bound and UB is upper bound.


### **Generate Pyomo Code**

In [18]:
response4 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please write a python pyomo code for this optimization problem. Use sample data where needed. Indicate where you use sample data."},
        {
            "role": "user",
            "content": response1.choices[0].message.content + response2.choices[0].message.content + response3.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 3
)

In [19]:
Markdown(response4.choices[0].message.content)

Here is your Pyomo code.

Please replace `sample_value` with your actual data:

```python
from pyomo.environ import *

# Create the concrete model
model = ConcreteModel()

# Define the variables
model.x = Var(RangeSet(1,6), domain=NonNegativeReals)

# Define the parameter (use your own sample value)
price = sample_value
cost_olefin = sample_value
cost_isobutane_recycle = sample_value
cost_acid = sample_value
cost_isobutane_makeup = sample_value

# Define the objective
model.obj = Objective(expr = price*(model.x[1]*model.x[2]) - cost_olefin*(model.x[1]) - cost_isobutane_recycle*(model.x[2]) - cost_acid*(model.x[3]) - cost_isobutane_makeup*(model.x[4]), sense=maximize)

# Define the constraints
model.con1 = Constraint(expr = 1.12 + 0.13167*model.x[5] - 0.00667*model.x[5]**2 == model.x[1])
model.con2 = Constraint(expr = 86.35 + 1.098*model.x[5] - 0.038*model.x[5]**2 + 0.325*(model.x[4] - 89) == model.x[2])
model.con3 = Constraint(expr = 35.82 - 0.222*model.x[3] == model.x[6])
model.con4 = Constraint(expr = -133 + 3*model.x[2] == model.x[3])

model.con5 = Constraint(expr = model.x[5] == (model.x[2] + model.x[4])/model.x[1])
model.con6 = Constraint(expr = model.x[4] == 93000*model.x[3]/(model.x[3] * model.x[6] + 1000*model.x[3]))
model.con7 = Constraint(expr = 1.22*model.x[1] == model.x[1] + model.x[4])

# Define the lower and upper bounds for each decision variables (use your own sample value)
model.x[1].setub(sample_value)
model.x[1].setlb(sample_value)
model.x[2].setub(sample_value)
model.x[2].setlb(sample_value)
model.x[3].setub(sample_value)
model.x[3].setlb(sample_value)
model.x[4].setub(sample_value)
model.x[4].setlb(sample_value)
model.x[5].setub(sample_value)
model.x[5].setlb(sample_value)
model.x[6].setub(sample_value)
model.x[6].setlb(sample_value)

# Solve the problem
solver = SolverFactory('glpk')
solver.solve(model)

# Print the results
print('Solution:')
for i in model.x:
    print('x[',i,']=',model.x[i].value)
print('Objective value= ', model.obj())
```

### **Run the code Generated by GPT4**

In [13]:
# Install Python API for AMPL
!pip install amplpy --upgrade

from amplpy import AMPL,ampl_notebook

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 23.7 MB/s eta 0:00:00


In [16]:
_ = load_dotenv(find_dotenv('second_knitro_license.env'))
my_knitro_license = os.environ['knitro_license']

ampl = ampl_notebook(
    modules=["coin"],  # modules to install
    license_uuid=my_knitro_license,  # license to use
)

Licensed to AMPL Community Edition License for <cevikmelis14@gmail.com>.


In [25]:
from pyomo.environ import *

# Create the concrete model
model = ConcreteModel()

# Define the variables
model.x = Var(RangeSet(1,6), domain=NonNegativeReals)

# Define the parameter (use your own sample value)
price = 0.63
cost_olefin = 5.04
cost_isobutane_recycle = 0.035
cost_acid = 10
cost_isobutane_makeup = 3.36
# Define the objective
model.obj = Objective(expr = price*(model.x[1]*model.x[2]) - cost_olefin*(model.x[1]) - cost_isobutane_recycle*(model.x[2]) - cost_acid*(model.x[3]) - cost_isobutane_makeup*(model.x[4]), sense=maximize)

# Define the constraints
model.con1 = Constraint(expr = 1.12 + 0.13167*model.x[5] - 0.00667*model.x[5]**2 == model.x[1])
model.con2 = Constraint(expr = 86.35 + 1.098*model.x[5] - 0.038*model.x[5]**2 + 0.325*(model.x[4] - 89) == model.x[2])
model.con3 = Constraint(expr = 35.82 - 0.222*model.x[3] == model.x[6])
model.con4 = Constraint(expr = -133 + 3*model.x[2] == model.x[3])

model.con5 = Constraint(expr = model.x[5] == (model.x[2] + model.x[4])/model.x[1])
model.con6 = Constraint(expr = model.x[4] == 93000*model.x[3]/(model.x[3] * model.x[6] + 1000*model.x[3]))
model.con7 = Constraint(expr = 1.22*model.x[1] == model.x[1] + model.x[4])

# Define the lower and upper bounds for each decision variables (use your own sample value)
model.x[1].setub(2000)
model.x[1].setlb(0)
model.x[2].setub(16000)
model.x[2].setlb(0)
model.x[3].setub(120)
model.x[3].setlb(0)
model.x[4].setub(2000)
model.x[4].setlb(0)
model.x[5].setub(12)
model.x[5].setlb(3)
model.x[6].setub(4)
model.x[6].setlb(1.2)

# Solve the problem
solver = SolverFactory('couenne')
solver.solve(model)

# Print the results
print('Solution:')
for i in model.x:
    print('x[',i,']=',model.x[i].value)
print('Objective value= ', model.obj())

  - termination condition: infeasible
  - message from solver: Couenne (/tmp/tmp423udbn4.pyomo.nl Feb 21 2023)\x3a Infeasible
ERROR:pyomo.common.numeric_types:evaluating object as numeric value: x[1]
    (object: <class 'pyomo.core.base.var.VarData'>)
No value for uninitialized NumericValue object x[1]


Solution:
x[ 1 ]= None
x[ 2 ]= None
x[ 3 ]= None
x[ 4 ]= None
x[ 5 ]= None
x[ 6 ]= None


ValueError: No value for uninitialized NumericValue object x[1]

### **Edit and Run the code for the mathematical model produced by GPT4 (Circumstantial)**

In [31]:
from pyomo.environ import *

model = ConcreteModel()

# Decision variables
model.x1 = Var(within=NonNegativeReals)
model.x2 = Var(within=NonNegativeReals)
model.x3 = Var(within=NonNegativeReals)
model.x4 = Var(within=NonNegativeReals)
model.x5 = Var(within=NonNegativeReals)
model.x6 = Var(within=NonNegativeReals)
model.y1 = Var(within=NonNegativeReals)
model.y2 = Var(within=NonNegativeReals)
model.y3 = Var(within=NonNegativeReals)
model.y4 = Var(within=NonNegativeReals)

# Objective
price = 0.63
cost_olefin = 5.04
cost_isobutane_recycle = 0.035
cost_acid = 10
cost_isobutane_makeup = 3.36
# Define the objective
model.obj = Objective(expr = price*(model.y1*model.y2) - cost_olefin*(model.x1) - cost_isobutane_recycle*(model.x2) - cost_acid*(model.x3) - cost_isobutane_makeup*(model.x4), sense=maximize)

# Constraints
model.Constraint1 = Constraint(expr = model.y1 <=1.02*model.x1 * (1.12 + 0.13167 * model.x5 - 0.00667 * model.x5**2))
model.Constraint11 = Constraint(expr = model.y1 >= 0.98*model.x1 * (1.12 + 0.13167 * model.x5 - 0.00667 * model.x5**2))

model.Constraint2 = Constraint(expr = model.y2 <= 1.01*(86.35 + 1.098 * model.x5 - 0.038 * model.x5**2 + 0.325 * (model.y4 - 89)))
model.Constraint22 = Constraint(expr = model.y2 >= 0.99*(86.35 + 1.098 * model.x5 - 0.038 * model.x5**2 + 0.325 * (model.y4 - 89)))

model.Constraint3 = Constraint(expr = model.x6 <= 1.05*(35.82 - 0.222 * model.y3))
model.Constraint33 = Constraint(expr = model.x6 >= 0.95*(35.82 - 0.222 * model.y3))

model.Constraint4 = Constraint(expr = model.y3 <= 1.1*(-133 + 3 * model.y2))
model.Constraint44 = Constraint(expr = model.y3 >= 0.9*(-133 + 3 * model.y2))

model.Constraint5 = Constraint(expr = model.x5 == (model.x2 + model.x4) / model.x1)
model.Constraint6 = Constraint(expr = model.y4 == (93000 * model.x3) / (model.x3 * model.x6 + 1000 * model.x3))
model.Constraint7 = Constraint(expr = 1.22 * model.y1 == model.x1 + model.x4)

# Bounds
# For each variables, replace lx and ux with its respective lower and upper value
model.x1.setub(2000)
model.x1.setlb(0)
model.x2.setub(16000)
model.x2.setlb(0)
model.x3.setub(120)
model.x3.setlb(0)
model.x4.setub(2000)
model.x4.setlb(0)
model.x5.setub(12)
model.x5.setlb(3)
model.x6.setub(4)
model.x6.setlb(1.2)


# Solve
solver = SolverFactory('couenne')
solver.solve(model)

# Print values
print("x1 =", model.x1())
print("x2 =", model.x2())
print("x3 =", model.x3())
print("x4 =", model.x4())
print("x5 =", model.x5())
print("x6 =", model.x6())
print("y1 =", model.y1())
print("y2=", model.y2())
print("y3=", model.y3())
print("y4 =", model.y4())
print("Profit =", model.obj())

x1 = 2000.0
x2 = 7865.48952873745
x3 = 1.364242052659392e-12
x4 = 2000.0
x5 = 4.932744764368723
x6 = 1.2
y1 = 3278.688524590164
y2= 102.18642451975784
y3= 156.2033462033462
y4 = 120.79427349431676
Profit = 193998.30605484644
